In [10]:
import numpy as np
from stavevec_utils import *
from time import time

In [28]:
N =24
K = 12
t0 = time()
basis =  build_basis_u64(N,K)
print('Time to build basis = ', time()-t0)
cache = SwapCache(basis)



Time to build basis =  17.35788917541504


In [25]:
t0 = time()

psi = init_singlet_product_state(basis,N) # creates product state (|01> - |10>) ^{N/2}
print('Time to build inital state = ', time()-t0)
psi_0 = psi.copy()

t0 = time()
apply_gate(cache, psi, 1, 2,np.pi/16) # apply e^{-i*(\pi/16) *S_{12}}|\psi>
print('Time to apply 1 gate (inital on the fly cache build) = ', time()-t0)

t0 = time()
apply_gate(cache, psi, 1, 2,np.pi/16) # apply e^{-i*(\pi/16) *S_{12}}|\psi>
print('Time to apply 1 gate (after cache built) = ', time()-t0)

Time to build inital state =  0.08089089393615723
Time to apply 1 gate (inital on the fly cache build) =  0.6926572322845459
Time to apply 1 gate (after cache built) =  0.013862133026123047


In [27]:
import numpy as np
import numba as nb

@nb.njit
def nCk(n: int, k: int) -> np.uint64:
    if k < 0 or k > n:
        return np.uint64(0)
    k = min(k, n - k)
    num = np.uint64(1)
    den = np.uint64(1)
    for i in range(1, k + 1):
        num *= np.uint64(n - (k - i))
        den *= np.uint64(i)
    return num // den

@nb.njit
def unrank_comb_to_mask(rank: np.uint64, N: int, K: int) -> np.uint64:
    """
    Return bitmask (uint64) with K ones among N bits.
    Unranking in lex-by-position order (smallest positions first).
    """
    mask = np.uint64(0)
    x = 0           # next position to consider
    k = K
    r = rank

    while k > 0:
        # Decide whether to place a 1 at position x
        # Count how many combinations if we skip position x:
        # i.e., choose k ones from remaining (N - (x+1)) positions
        c = nCk(N - (x + 1), k)
        if r < c:
            # skip x (bit 0), move on
            x += 1
        else:
            # take x (bit 1)
            r -= c
            mask |= (np.uint64(1) << np.uint64(x))
            x += 1
            k -= 1

    return mask

@nb.njit(parallel=True)
def build_basis_u64_numba(N: int, K: int) -> np.ndarray:
    M = nCk(N, K)
    basis = np.empty(M, dtype=np.uint64)
    for r in nb.prange(M):
        basis[r] = unrank_comb_to_mask(np.uint64(r), N, K)
    return basis


In [67]:
N, K = 32, 16
t0 = time()
basis = build_basis_u64_numba(N, K)
print('Time to build basis = ', time()-t0)
t0 = time()
#print(basis)
basis.sort()
print('Time to sort basis = ', time()-t0)
#print(basis)

Time to build basis =  0.08265995979309082
Time to sort basis =  0.03292417526245117


In [68]:
#B =  build_basis_u64(N,K)
print(len(basis),binom(N,K))

671324 601080390.0


In [59]:
import scipy
from scipy.special import binom

In [69]:
2**18

262144

In [70]:
binom(24,12)

2704156.0

In [73]:
2**18

262144

In [78]:
binom(12,6)

924.0

In [76]:
binom(24,12)

2704156.0

In [80]:
6**6

46656